In [1]:
!pip install numpy pandas tensorflow matplotlib


In [2]:
import requests

# Download Shakespeare text file
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)

# Save it to a local file
with open("shakespeare.txt", "w") as file:
    file.write(response.text)

print("Dataset downloaded successfully!")


Dataset downloaded successfully!


In [3]:
# Load the dataset
with open("shakespeare.txt", "r") as file:
    text = file.read()

# Preprocess the text
text = text.lower()

# Display the first 500 characters
print(text[:500])


first citizen:
before we proceed any further, hear me speak.

all:
speak, speak.

first citizen:
you are all resolved rather to die than to famish?

all:
resolved. resolved.

first citizen:
first, you know caius marcius is chief enemy to the people.

all:
we know't, we know't.

first citizen:
let us kill him, and we'll have corn at our own price.
is't a verdict?

all:
no more talking on't; let it be done: away, away!

second citizen:
one word, good citizens.

first citizen:
we are accounted poor


In [4]:
# Create a mapping from characters to integers
vocab = sorted(set(text))
char_to_index = {char: idx for idx, char in enumerate(vocab)}
index_to_char = {idx: char for idx, char in enumerate(vocab)}

# Encode the entire text as integers
text_as_int = [char_to_index[c] for c in text]

print("Vocabulary size:", len(vocab))
print("First 20 characters as integers:", text_as_int[:20])


Vocabulary size: 39
First 20 characters as integers: [18, 21, 30, 31, 32, 1, 15, 21, 32, 21, 38, 17, 26, 10, 0, 14, 17, 18, 27, 30]


In [5]:
import numpy as np

# Sequence length
seq_length = 100
sequences = []

for i in range(0, len(text_as_int) - seq_length, seq_length):
    seq = text_as_int[i:i + seq_length + 1]
    sequences.append(seq)

# Convert to numpy array
sequences = np.array(sequences)

print(f"Total sequences: {len(sequences)}")


Total sequences: 11153


In [6]:
X = []
Y = []

for seq in sequences:
    X.append(seq[:-1])
    Y.append(seq[-1])

X = np.array(X)
Y = np.array(Y)

# One-hot encode the output labels
from tensorflow.keras.utils import to_categorical
Y = to_categorical(Y, num_classes=len(vocab))

print("Input shape:", X.shape)
print("Output shape:", Y.shape)


Input shape: (11153, 100)
Output shape: (11153, 39)


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding


In [8]:
model = Sequential()

# Embedding layer
model.add(Embedding(input_dim=len(vocab), output_dim=64, input_length=seq_length))

# LSTM layers
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(256))

# Output layer
model.add(Dense(len(vocab), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Train the model
history = model.fit(X, Y, epochs=10, batch_size=64, validation_split=0.2)


Epoch 1/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 146s 1s/step - accuracy: 0.1623 - loss: 3.0458 - val_accuracy: 0.2102 - val_loss: 2.7919
Epoch 2/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 144s 1s/step - accuracy: 0.2324 - loss: 2.7098 - val_accuracy: 0.3021 - val_loss: 2.4484
Epoch 3/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 144s 1s/step - accuracy: 0.2974 - loss: 2.4187 - val_accuracy: 0.3021 - val_loss: 2.3619
Epoch 4/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 145s 1s/step - accuracy: 0.3259 - loss: 2.3099 - val_accuracy: 0.3420 - val_loss: 2.2820
Epoch 5/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 201s 1s/step - accuracy: 0.3543 - loss: 2.1978 - val_accuracy: 0.3523 - val_loss: 2.2323
Epoch 6/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 202s 1s/step - accuracy: 0.3753 - loss: 2.1116 - val_accuracy: 0.3321 - val_loss: 2.2033
Epoch 7/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 201s 1s/step - accuracy: 0.4029 - loss: 2.0190 - val_accuracy: 0.3631 - val_loss: 2.1728
Epoch 8/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 202s 1s/step - accuracy: 0.4307 - loss: 1.8811 - val_accu

In [11]:
model.save("text_gen_lstm.h5")


In [12]:
from google.colab import files
files.download("text_gen_lstm.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
import random

def generate_text(model, seed_text, length=300, temperature=1.0):
    result = seed_text
    for _ in range(length):
        # Tokenize the seed text
        seed_int = [char_to_index[c] for c in seed_text]
        seed_int = np.expand_dims(seed_int, 0)

        # Predict the next character
        predictions = model.predict(seed_int)[0]

        # Apply temperature to add randomness
        predictions = np.log(predictions) / temperature
        exp_preds = np.exp(predictions)
        predictions = exp_preds / np.sum(exp_preds)

        # Sample next character
        next_index = np.random.choice(len(vocab), p=predictions)
        next_char = index_to_char[next_index]

        # Append to result
        result += next_char
        seed_text = seed_text[1:] + next_char

    return result


In [14]:
# Example: Generate text with different temperatures
seed_text = "to be or not to be"
print("\nGenerated Text (Temperature 0.5):\n")
print(generate_text(model, seed_text, length=500, temperature=0.5))

print("\nGenerated Text (Temperature 1.0):\n")
print(generate_text(model, seed_text, length=500, temperature=1.0))



Generated Text (Temperature 0.5):

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 479ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━